<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [44]:
!pip3 install twilio

In [45]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [46]:
query = 'London'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=39b56f497c7f4336bd841816240602 &q=London&days=1&aqi=no&alerts=no'

In [47]:
response = requests.get(url_clima).json()

In [48]:
response

{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.52,
  'lon': -0.11,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1707200078,
  'localtime': '2024-02-06 6:14'},
 'current': {'last_updated_epoch': 1707199200,
  'last_updated': '2024-02-06 06:00',
  'temp_c': 12.0,
  'temp_f': 53.6,
  'is_day': 0,
  'condition': {'text': 'Overcast',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png',
   'code': 1009},
  'wind_mph': 16.1,
  'wind_kph': 25.9,
  'wind_degree': 240,
  'wind_dir': 'WSW',
  'pressure_mb': 1011.0,
  'pressure_in': 29.85,
  'precip_mm': 0.01,
  'precip_in': 0.0,
  'humidity': 77,
  'cloud': 100,
  'feelslike_c': 9.5,
  'feelslike_f': 49.0,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 23.5,
  'gust_kph': 37.8},
 'forecast': {'forecastday': [{'date': '2024-02-06',
    'date_epoch': 1707177600,
    'day': {'maxtemp_c': 12.2,
     'maxtemp_f': 53.9,
     'mintemp_c': 9.6,
     

In [49]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [50]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [51]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [52]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1707177600,
 'time': '2024-02-06 00:00',
 'temp_c': 11.4,
 'temp_f': 52.6,
 'is_day': 0,
 'condition': {'text': 'Overcast ',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png',
  'code': 1009},
 'wind_mph': 15.4,
 'wind_kph': 24.8,
 'wind_degree': 238,
 'wind_dir': 'WSW',
 'pressure_mb': 1015.0,
 'pressure_in': 29.97,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'snow_cm': 0.0,
 'humidity': 67,
 'cloud': 95,
 'feelslike_c': 8.8,
 'feelslike_f': 47.8,
 'windchill_c': 8.8,
 'windchill_f': 47.8,
 'heatindex_c': 11.4,
 'heatindex_f': 52.6,
 'dewpoint_c': 5.6,
 'dewpoint_f': 42.0,
 'will_it_rain': 0,
 'chance_of_rain': 0,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 10.0,
 'vis_miles': 6.0,
 'gust_mph': 23.1,
 'gust_kph': 37.2,
 'uv': 1.0,
 'short_rad': 0.0,
 'diff_rad': 0.0}

In [53]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2024-02-06'

In [54]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [55]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [56]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [57]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [58]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


In [59]:
datos[0]

('2024-02-06', 0, 'Overcast ', 11.4, 0, 0)

In [60]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2024-02-06,0,Overcast,11.4,0,0
1,2024-02-06,1,Overcast,11.4,0,0
2,2024-02-06,2,Overcast,11.5,0,0
3,2024-02-06,3,Cloudy,11.5,0,0
4,2024-02-06,4,Cloudy,11.6,0,0
5,2024-02-06,5,Partly Cloudy,11.6,0,0
6,2024-02-06,6,Overcast,12.0,1,76
7,2024-02-06,7,Overcast,11.3,0,0
8,2024-02-06,8,Patchy rain nearby,11.0,0,65
9,2024-02-06,9,Patchy rain nearby,11.3,1,82


In [61]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [62]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
6,2024-02-06,6,Overcast,12.0,1,76
9,2024-02-06,9,Patchy rain nearby,11.3,1,82
17,2024-02-06,17,Patchy rain nearby,11.8,1,100
18,2024-02-06,18,Light drizzle,12.0,1,100
19,2024-02-06,19,Light drizzle,11.7,1,100
20,2024-02-06,20,Light rain,11.5,1,100
21,2024-02-06,21,Moderate rain,11.0,1,100
22,2024-02-06,22,Moderate rain,9.8,1,100
23,2024-02-06,23,Light rain,6.5,1,100


In [64]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [65]:
df['Fecha'][0]

'2024-02-06'

In [66]:
df_rain

,Condicion
Hora,
9,Patchy rain nearby
17,Patchy rain nearby
18,Light drizzle
19,Light drizzle
20,Light rain
21,Moderate rain


In [67]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2024-02-06 en London es : \n\n\n                Condicion\nHora                    \n9     Patchy rain nearby\n17    Patchy rain nearby\n18         Light drizzle\n19         Light drizzle\n20            Light rain\n21         Moderate rain'

In [68]:
PHONE_NUMBER

'+12342399301'

# Mensaje SMS desde Twilio

In [69]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+51992634890'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM55ea14b967e4c9609343ee2f0b872c3d


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />